In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - make_table.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Make a table of results for the paper
'''

__author__ = "James Lane"

In [ ]:
### Imports

# Basic
import os, sys, copy
import numpy as np

# Matplotlib and plotting 
import matplotlib.pyplot as plt

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import util as putil

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, Pathing, Loading, Data Preparation

In [ ]:
# %load ../../src/nb_modules/keywords_pathing_loading_data_prep.py
## Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','LOGG_MIN','LOGG_MAX','FEH_MIN','FEH_MAX',
            'FEH_MIN_GSE','FEH_MAX_GSE','DF_VERSION','KSF_VERSION','NPROCS',
            'RO','VO','ZO']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    logg_min,logg_max,feh_min,feh_max,feh_min_gse,feh_max_gse,df_version,\
    ksf_version,nprocs,ro,vo,zo = putil.parse_config_dict(cdict,keywords)
logg_range = [logg_min,logg_max]
feh_range = [feh_min,feh_max]
feh_range_gse = [feh_min_gse,feh_max_gse]
feh_range_all = [feh_min,feh_max_gse]
# feh_range_fit = copy.deepcopy( # Need to choose here


## Pathing
fit_paths = putil.prepare_paths(base_dir,apogee_dr,apogee_results_vers,gaia_dr,
                                df_version,ksf_version)
data_dir,version_dir,ga_dir,gap_dir,df_dir,ksf_dir,fit_dir = fit_paths

## Filenames
fit_filenames = putil.prepare_filenames(ga_dir,gap_dir,feh_range_gse)
apogee_SF_filename,apogee_effSF_filename,apogee_effSF_mask_filename,\
    iso_grid_filename,clean_kinematics_filename = fit_filenames

## File loading and data preparation
fit_stuff,other_stuff = putil.prepare_fitting(fit_filenames,
    [ndmod,dmod_min,dmod_max],ro,zo,return_other=True)
apogee_effSF_mask,dmap,iso_grid,jkmins,dmods,ds,effsel_grid,apof,\
    allstar_nomask,orbs_nomask = fit_stuff
Rgrid,phigrid,zgrid = effsel_grid
# apogee_SF,apogee_effSF_grid_inclArea,apogee_effSF_grid_inclArea_Jac = other_stuff

# ## Load the distribution functions
# df_filename = df_dir+'dfs.pkl'
# betas = [0.3,0.9]
# dfs = putil.load_distribution_functions(df_filename, betas)

# ## Load the APOGEE field information, can also similarly load 
# ## '...apogee_field_glons.npy', '...apogee_field_glats.npy', 
# ## '...apogee_field_location_ids.npy'
# apogee_fields = np.load(ga_dir+'apogee_fields.npy')

### Global Parameters

In [ ]:
# %load ../../src/nb_modules/global_fitting_params.py
## general kwargs
verbose = True

## HaloFit kwargs (ordering follows HaloFit.__init__)
# allstar and orbs loaded in prep cell
init = None
init_type = 'ML'
# fit_type provided at runtime
mask_disk = True
mask_halo = True
# densfunc, selec provided at runtime
# effsel, effsel_grid, effsel_mask, dmods loaded in prep cell
nwalkers = 100
nit = int(2e3)
ncut = int(1e3)
# usr_log_prior provided at runtime
n_mass = 5000 # int(nwalkers*(nit-ncut))
int_r_range = [2.,70.]
iso = None # Will read from iso_grid_filename
# iso_filename, jkmins loaded in prep cell
# feh_range provided at runtime
# logg_range loaded in config cell
# fit_dir, gap_dir, ksf_dir loaded in prep cell
# version provided at runtime
# ro, vo, zo loaded in config cell

hf_kwargs = {## HaloFit parameters
             'allstar':allstar_nomask,
             'orbs':orbs_nomask,
             'init':init,
             'init_type':init_type,
             # 'fit_type':fit_type, # provided at runtime
             'mask_disk':mask_disk,
             'mask_halo':mask_halo,
             ## _HaloFit parameters
             # 'densfunc':densfunc, # provided at runtime
             # 'selec':selec, # provided at runtime
             'effsel':apof,
             'effsel_mask':apogee_effSF_mask,
             'effsel_grid':effsel_grid,
             'dmods':dmods,
             'nwalkers':nwalkers,
             'nit':nit,
             'ncut':ncut,
             # 'usr_log_prior':usr_log_prior, # provided at runtime
             'n_mass':n_mass,
             'int_r_range':int_r_range,
             'iso':iso,
             'iso_filename':iso_grid_filename,
             'jkmins':jkmins,
             # 'feh_range':feh_range, # provided at runtime
             'logg_range':logg_range,
             'fit_dir':fit_dir,
             'gap_dir':gap_dir,
             'ksf_dir':ksf_dir,
             # 'version':version, # provided at runtime
             'verbose':verbose,
             'ro':ro,
             'vo':vo,
             'zo':zo}

## pmass.fit() function kwargs
# nprocs set in config file
force_fit = True
mle_init = True
just_mle = False
return_walkers = True
optimizer_method = 'Powell'
mass_int_type = 'spherical_grid'
batch_masses = True
make_ml_aic_bic = True
calculate_masses = True
post_optimization = True
mcmc_diagnostic = True

fit_kwargs = {# 'nprocs':nprocs, # Normally given at runtime 
              'force_fit':force_fit,
              'mle_init':mle_init,
              'just_mle':just_mle,
              'return_walkers':return_walkers,
              'optimizer_method':optimizer_method,
              'mass_int_type':mass_int_type,
              'batch_masses':batch_masses,
              'make_ml_aic_bic':make_ml_aic_bic,
              'calculate_masses':calculate_masses,
              'post_optimization':post_optimization,
              'mcmc_diagnostic':mcmc_diagnostic,
              }

## Functions

#### Structural table

In [ ]:
def make_gse_structural_table(densfuncs,versions,feh_range,fit_type,
                              hf_kwargs={}):
    '''make_gse_structural_table:
    
    Make a table of the results of the structural parameters of the models
    for the GS/E fits
    
    Args:
        densfuncs (list) - List of densfuncs to plot
        versions (list) - List of versions corresponding to densfuncs
        feh_range (list) - 2-element list of feh_min and feh_max
        fit_type (str) - String corresponding to type of halo fit
        hf_kwargs (dict) - Dictionary of HaloFit class keywords [default {}]
    
    Returns:
        table (str) - Table text
    '''
    # Commonly used sequences
    SEP=' & '
    END=' \\\\ \n'
    HLINE = '\hline \n'
    
    # Sanity
    assert isinstance(versions,str) or (len(densfuncs) == len(versions))
    
    # phi and theta units
    phi_in_degrees = True
    theta_in_degrees = True
    if phi_in_degrees:
        phi_unit = '$[\mathrm{deg}]$'
    else:
        phi_unit = '$[\mathrm{rad}]$'
    if theta_in_degrees:
        theta_unit = '$[\mathrm{deg}]$'
    else:  
        theta_unit = '$[\mathrm{rad}]$'

    # Initialize the table and write column information
    ncol = 13
    table = ''
    selec_names_nice = {'eLz':'$e-L_\mathrm{z}$',
                        'AD':'AD',
                        'JRLz':'$\sqrt{J_\mathrm{R}}-L_\mathrm{z}$'}
    column_names = ['Name','$\\alpha_{1}$','$\\alpha_{2}$','$\\alpha_{3}$',
                    '$r_{1}$','$r_{2}$','$p$','$q$','$\\theta$',
                    '$\\eta$','$\\phi$','$f_\mathrm{disk}$','Mass']
    column_units = ['','','','','$[\mathrm{kpc}]$','$[\mathrm{kpc}]$','','',
                    theta_unit,'',phi_unit,'','$[10^{8}~M_{\odot}]$']
    names_txt = ''
    for i in range(len(column_names)):
        names_txt += column_names[i]
        if i == len(column_names)-1:
            names_txt += END
        else:
            names_txt += SEP
    units_txt = ''
    for i in range(len(column_units)):
        units_txt += column_units[i]
        if i == len(column_units)-1:
            units_txt += END
        else:
            units_txt += SEP
    
    table += HLINE+names_txt+units_txt+HLINE
    
    
    # First section is e-Lz
    selec = 'eLz'
    section1_txt = make_structural_table_section(densfuncs,fit_type,versions,
                                                 selec,feh_range,hf_kwargs,
                                                 theta_in_degrees=theta_in_degrees,
                                                 phi_in_degrees=phi_in_degrees)
    selec_txt = '\multicolumn{'+str(ncol)+'}{c}{'+selec_names_nice[selec]+'}'
    table += selec_txt+END+HLINE
    table += section1_txt+HLINE
    
    # Second section is AD
    selec = 'AD'
    section2_txt = make_structural_table_section(densfuncs,fit_type,versions,
                                                 selec,feh_range,hf_kwargs,
                                                 theta_in_degrees=theta_in_degrees,
                                                 phi_in_degrees=phi_in_degrees)
    selec_txt = '\multicolumn{'+str(ncol)+'}{c}{'+selec_names_nice[selec]+'}'
    table += selec_txt+END+HLINE
    table += section2_txt+HLINE
    
    # Third section is JRLz
    selec = 'JRLz'
    section3_txt = make_structural_table_section(densfuncs,fit_type,versions,
                                                 selec,feh_range,hf_kwargs,
                                                 theta_in_degrees=theta_in_degrees,
                                                 phi_in_degrees=phi_in_degrees)
    selec_txt = '\multicolumn{'+str(ncol)+'}{c}{'+selec_names_nice[selec]+'}'
    table += selec_txt+END+HLINE
    table += section3_txt+HLINE
    
    return table

def make_halo_structural_table(densfuncs,versions,feh_range,fit_type,
                              hf_kwargs={}):
    '''make_halo_structural_table:
    
    Make a table of the results of the structural parameters of the models
    for the whole-halo fits
    
    Args:
        densfuncs (list) - List of densfuncs to plot
        versions (list) - List of versions corresponding to densfuncs
        feh_range (list) - 2-element list of feh_min and feh_max
        fit_type (str) - String corresponding to type of halo fit
        hf_kwargs (dict) - Dictionary of HaloFit class keywords [default {}]
    
    Returns:
        table (str) - Table text
    '''
    # Commonly used sequences
    SEP=' & '
    END=' \\\\ \n'
    HLINE = '\hline \n'
    
    # Sanity
    assert isinstance(versions,str) or (len(densfuncs) == len(versions))
    
    # phi and theta units
    phi_in_degrees = True
    theta_in_degrees = True
    if phi_in_degrees:
        phi_unit = '$[\mathrm{deg}]$'
    else:
        phi_unit = '$[\mathrm{rad}]$'
    if theta_in_degrees:
        theta_unit = '$[\mathrm{deg}]$'
    else:  
        theta_unit = '$[\mathrm{rad}]$'

    # Initialize the table and write column information
    ncol = 13
    table = ''
    column_names = ['Name','$\\alpha_{1}$','$\\alpha_{2}$','$\\alpha_{3}$',
                    '$r_{1}$','$r_{2}$','$p$','$q$','$\\theta$',
                    '$\\eta$','$\\phi$','$f_\mathrm{disk}$','Mass']
    column_units = ['','','','','$[\mathrm{kpc}]$','$[\mathrm{kpc}]$','','',
                    theta_unit,'',phi_unit,'',
                    '$[10^{8}~M_{\odot}]$']
    names_txt = ''
    for i in range(len(column_names)):
        names_txt += column_names[i]
        if i == len(column_names)-1:
            names_txt += END
        else:
            names_txt += SEP
    units_txt = ''
    for i in range(len(column_units)):
        units_txt += column_units[i]
        if i == len(column_units)-1:
            units_txt += END
        else:
            units_txt += SEP
    
    table += HLINE+names_txt+units_txt+HLINE
    
    # Only one section
    section1_txt = make_structural_table_section(densfuncs,fit_type,versions,
                                                 None,feh_range,hf_kwargs,
                                                 theta_in_degrees=theta_in_degrees,
                                                 phi_in_degrees=phi_in_degrees)
    # selec_txt = '\multicolumn{'+str(ncol)+'}{c}{'+selec_names_nice[selec]+'}'
    # table += selec_txt+END+HLINE
    table += section1_txt+HLINE
    
    return table

def make_structural_table_section(densfuncs,fit_type,versions,selec,feh_range,
                                  hf_kwargs,theta_in_degrees=True,
                                  phi_in_degrees=True,mass_in_log=True):
    '''make_structural_table_section:
    
    Make a section for the table of structural results for the paper.
    
    Args:
        densfuncs
        fit_type
        versions
        selec
        feh_range
        hf_kwargs
        theta_in_degrees
        phi_in_degrees
        mass_in_log
    
    Returns:
        section (str) - Section text
    '''
    # Commonly used sequences
    SEP=' & '
    END=' \\\\'
    
    # Parameters
    quantiles = [0.16,0.5,0.84]
    # Scaling for theta and phi and mass
    theta_scale = 1.
    theta_n_round = 2
    phi_scale = 1.
    phi_n_round = 2
    mass_scale = 1e8 # 10^8 solar masses
    mass_n_round = 2
    if theta_in_degrees:
        theta_scale = 180./np.pi
        theta_n_round = 1
    if phi_in_degrees:
        phi_scale = 180./np.pi
        phi_n_round = 1
    if mass_in_log:
        mass_scale = 1.

    # Each map is length-12, tells which parameter index is p,q,theta,eta,phi
    shape_param_map = np.array([-5,-4,-3,-2,-1])
    theta_indx = 2
    phi_indx = 4
    # alpha_1, alpha_2, alpha_3, r1, r2
    struct_param_map = {'triaxial_single_angle_zvecpa':[0,-1,-1,-1,-1],
                        'triaxial_single_cutoff_zvecpa':[0,-1,-1, 1,-1],
                        'triaxial_broken_angle_zvecpa':[0, 1,-1, 2,-1],
                        'triaxial_double_broken_angle_zvecpa':[0, 1, 2, 3, 4]}
    densfunc_names = {'triaxial_single_angle_zvecpa':'SPL',
                      'triaxial_single_cutoff_zvecpa':'SC',
                      'triaxial_broken_angle_zvecpa':'BPL',
                      'triaxial_double_broken_angle_zvecpa':'DBPL'
                      }
    if isinstance(versions,str):
        versions = np.repeat(versions,len(densfuncs))
    
    # Begin section
    section = ''
    for i in range(len(densfuncs)):
        this_line = ''
        
        # Load the results through the HaloFit class
        hf = pmass.HaloFit(densfunc=densfuncs[i], fit_type=fit_type, 
                           version=versions[i], selec=selec, 
                           feh_range=feh_range, **hf_kwargs)
        hf.get_results()
        hf.get_loglike_ml_aic_bic()
        samples = pdens.denormalize_parameters(hf.samples,hf.densfunc)
        
        # Set name
        densfunc_name = densfuncs[i].__name__
        isdisk = False
        if 'plusexpdisk' in densfunc_name:
            isdisk = True
            name = densfunc_name.split('_plusexpdisk')[0]
            abbrv = densfunc_names[name]+'+D'
        else:
            name = str(densfunc_name)
            abbrv = densfunc_names[name]
        this_line += (abbrv+SEP)
        
        # Quantiles
        qs = np.quantile(samples,quantiles,axis=0)
        median = qs[1]
        lowe = qs[1]-qs[0]
        highe = qs[2]-qs[1]
        
        # Loop over the 5 structure parameters: a1,a2,a2,r1,r2
        for j in range(5):
            # Does this position need to be blank for this profile?
            indx = struct_param_map[name][j]
            if indx == -1:
                this_line += '$-$'+SEP
            else:
                if median[indx] > 100:
                    med_txt = '\infty'
                    high_txt = '0'
                    low_txt = '0'
                else:
                    med_txt = str(round(median[indx],2))
                    high_txt = str(round(highe[indx],2))
                    low_txt = str(round(lowe[indx],2))
                medpm_txt = '$'+med_txt+'^{+'+high_txt+'}_{-'+low_txt+'}$'
                this_line += (medpm_txt+SEP)
            
        # Loop over shape parameters: p,q,theta,eta,phi
        
        for j in range(5):
            indx = shape_param_map[j]-int(isdisk)
            # Handle theta and phi
            if j == theta_indx:
                medpm_txt = '$'+str(round(median[indx]*theta_scale,theta_n_round))+\
                        '^{+'+str(round(highe[indx]*theta_scale,theta_n_round))+\
                        '}_{-'+str(round(lowe[indx]*theta_scale,theta_n_round))+'}$'
            elif j == phi_indx:
                medpm_txt = '$'+str(round(median[indx]*phi_scale,phi_n_round))+\
                        '^{+'+str(round(highe[indx]*phi_scale,phi_n_round))+\
                        '}_{-'+str(round(lowe[indx]*phi_scale,phi_n_round))+'}$'
            # Handle the rest
            else:
                medpm_txt = '$'+str(round(median[indx],2))+\
                        '^{+'+str(round(highe[indx],2))+\
                        '}_{-'+str(round(lowe[indx],2))+'}$'
            this_line += (medpm_txt+SEP)
        
        # Do the disk contamination fraction
        if isdisk:
            medpm_txt = '$'+str(round(median[-1],2))+\
                    '^{+'+str(round(highe[-1],2))+\
                    '}_{-'+str(round(lowe[-1],2))+'}$'
            this_line += (medpm_txt+SEP)
        else:
            this_line += ('$-$'+SEP)
        
        # Do the mass
        if mass_in_log:
            qs_mass = np.quantile(np.log10(hf.masses),quantiles)/mass_scale
        else:
            qs_mass = np.quantile(hf.masses,quantiles)/mass_scale
        median_mass = qs_mass[1]
        lowe_mass = qs_mass[1]-qs_mass[0]
        highe_mass = qs_mass[2]-qs_mass[1]
        medpm_txt = '$'+str(round(median_mass,2))+\
                    '^{+'+str(round(highe_mass,2))+\
                    '}_{-'+str(round(lowe_mass,2))+'}$'
        this_line += (medpm_txt)
        
        this_line += END+' \n'
        section += this_line
        
    return section



#### Likelihood table

In [ ]:
def make_gse_likelihood_table(densfuncs,versions,feh_range,fit_type,
                              ml_type='post',hf_kwargs={}):
    '''make_gse_likelihood_table:
    
    Make a table of the maximum likelihoods, AIC, BIC of the models
    for the GS/E fits
    
    Args:
        densfuncs (list) - List of densfuncs to plot
        versions (list) - List of versions corresponding to densfuncs
        feh_range (list) - 2-element list of feh_min and feh_max
        fit_type (str) - String corresponding to type of halo fit
        hf_kwargs (dict) - Dictionary of HaloFit class keywords [default {}]
    
    Returns:
        table (str) - Table text
    '''
    # Commonly used sequences
    SEP=' & '
    END=' \\\\ \n'
    HLINE = '\hline \n'
    
    # Sanity
    assert isinstance(versions,str) or (len(densfuncs) == len(versions))
    
    # Initialize the table and write column information
    ncol = 4
    table = ''
    selec_names_nice = {'eLz':'$e-L_\mathrm{z}$',
                        'AD':'AD',
                        'JRLz':'$\sqrt{J_\mathrm{R}}-L_\mathrm{z}$'}
    column_names = ['Name','$\mathrm{max}(\mathcal{L})$','AIC','BIC']
    column_units = ['','','','']
    names_txt = ''
    for i in range(len(column_names)):
        names_txt += column_names[i]
        if i == len(column_names)-1:
            names_txt += END
        else:
            names_txt += SEP
    units_txt = ''
    # for i in range(len(column_units)):
    #     units_txt += column_units[i]
    #     if i == len(column_units)-1:
    #         units_txt += END
    #     else:
    #         units_txt += SEP
    
    table += HLINE+names_txt+units_txt+HLINE
    
    # First section is e-Lz
    selec = 'eLz'
    section1_txt = make_likelihood_table_section(densfuncs,fit_type,versions,
                                                 selec,feh_range,ml_type,
                                                 hf_kwargs)
    selec_txt = '\multicolumn{'+str(ncol)+'}{c}{'+selec_names_nice[selec]+'}'
    table += selec_txt+END+HLINE
    table += section1_txt+HLINE
    
    # Second section is AD
    selec = 'AD'
    section2_txt = make_likelihood_table_section(densfuncs,fit_type,versions,
                                                 selec,feh_range,ml_type,
                                                 hf_kwargs)
    selec_txt = '\multicolumn{'+str(ncol)+'}{c}{'+selec_names_nice[selec]+'}'
    table += selec_txt+END+HLINE
    table += section2_txt+HLINE
    
    # Third section is JRLz
    selec = 'JRLz'
    section3_txt = make_likelihood_table_section(densfuncs,fit_type,versions,
                                                 selec,feh_range,ml_type,
                                                 hf_kwargs)
    selec_txt = '\multicolumn{'+str(ncol)+'}{c}{'+selec_names_nice[selec]+'}'
    table += selec_txt+END+HLINE
    table += section3_txt+HLINE
    
    return table

def make_halo_likelihood_table(densfuncs,versions,feh_range,fit_type,
                              ml_type='post',hf_kwargs={}):
    '''make_halo_likelihood_table:
    
    Make a table of the maximum likelihoods, AIC, BIC of the models
    for the GS/E fits
    
    Args:
        densfuncs (list) - List of densfuncs to plot
        versions (list) - List of versions corresponding to densfuncs
        feh_range (list) - 2-element list of feh_min and feh_max
        fit_type (str) - String corresponding to type of halo fit
        ml_type (str) - Type of maximum likelihood to get for the likelihood
            -based parameters. Can be 'mcmc_ml' (from chain), 'init' (initial 
            optimization) or 'post' (final optimization)
        hf_kwargs (dict) - Dictionary of HaloFit class keywords [default {}]
    
    Returns:
        table (str) - Table text
    '''
    # Commonly used sequences
    SEP=' & '
    END=' \\\\ \n'
    HLINE = '\hline \n'
    
    # Sanity
    assert isinstance(versions,str) or (len(densfuncs) == len(versions))
    
    # Initialize the table and write column information
    ncol = 4
    table = ''
    column_names = ['Name','$\mathrm{max}(\mathcal{L})$','AIC','BIC']
    column_units = ['','','','']
    names_txt = ''
    for i in range(len(column_names)):
        names_txt += column_names[i]
        if i == len(column_names)-1:
            names_txt += END
        else:
            names_txt += SEP
    units_txt = ''
    # for i in range(len(column_units)):
    #     units_txt += column_units[i]
    #     if i == len(column_units)-1:
    #         units_txt += END
    #     else:
    #         units_txt += SEP
    
    table += HLINE+names_txt+units_txt+HLINE
    
    
    # First section is e-Lz
    section1_txt = make_likelihood_table_section(densfuncs,fit_type,versions,
                                                 None,feh_range,ml_type,
                                                 hf_kwargs)
    # selec_txt = '\multicolumn{'+str(ncol)+'}{c}{'+selec_names_nice[selec]+'}'
    # table += selec_txt+END+HLINE
    table += section1_txt+HLINE
    
    return table

def make_likelihood_table_section(densfuncs,fit_type,versions,selec,feh_range,
                                  ml_type,hf_kwargs):
    '''make_likelihood_table_section:
    
    Make a section for the table of likelihood results for the paper.
    
    Args:
        densfuncs
        fit_type
        versions
        selec
        feh_range
        hf_kwargs
    
    Returns:
        section (str) - Section text
    '''
    # Commonly used sequences
    SEP=' & '
    END=' \\\\'
    
    # Parameters
    densfunc_names = {'triaxial_single_angle_zvecpa':'SPL',
                      'triaxial_single_cutoff_zvecpa':'SC',
                      'triaxial_broken_angle_zvecpa':'BPL',
                      'triaxial_double_broken_angle_zvecpa':'DBPL'
                      }
    if isinstance(versions,str):
        versions = np.repeat(versions,len(densfuncs))
    
    # Begin section
    section = ''
    for i in range(len(densfuncs)):
        this_line = ''
        
        # Load the results through the HaloFit class
        hf = pmass.HaloFit(densfunc=densfuncs[i], fit_type=fit_type, 
                           version=versions[i], selec=selec, 
                           feh_range=feh_range, **hf_kwargs)
        hf.get_results()
        hf.get_loglike_ml_aic_bic()
        
        # Set name
        densfunc_name = densfuncs[i].__name__
        isdisk = False
        if 'plusexpdisk' in densfunc_name:
            isdisk = True
            name = densfunc_name.split('_plusexpdisk')[0]
            abbrv = densfunc_names[name]+'+D'
        else:
            name = str(densfunc_name)
            abbrv = densfunc_names[name]
        this_line += (abbrv+SEP)
        
        ml,aic,bic = hf.calculate_ml_aic_bic(ml_type='post')
        
        if i==0:
            ml_norm = copy.deepcopy(ml)
            aic_norm = copy.deepcopy(aic)
            bic_norm = copy.deepcopy(bic)
            
        this_line += (str(round(ml-ml_norm,1))+SEP)
        this_line += (str(round(aic-aic_norm,1))+SEP)
        this_line += (str(round(bic-bic_norm,1)))
        this_line += END+' \n'
        
        section += this_line
        
    return section

## Make the structural tables

In [ ]:
densfuncs = [pdens.triaxial_single_angle_zvecpa,
             pdens.triaxial_single_angle_zvecpa_plusexpdisk,
             pdens.triaxial_single_cutoff_zvecpa,
             pdens.triaxial_single_cutoff_zvecpa_plusexpdisk,
             pdens.triaxial_broken_angle_zvecpa,
             pdens.triaxial_broken_angle_zvecpa_plusexpdisk,
            #]
             pdens.triaxial_double_broken_angle_zvecpa,
             pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk]
versions = '100w_1e4n'
fit_type = 'gse'
feh_range_fit = copy.deepcopy(feh_range_gse)
selec = 'eLz'

hf = pmass.HaloFit(densfunc=densfuncs[0], fit_type=fit_type, 
                           version=versions[0], selec=selec, 
                           feh_range=feh_range, **hf_kwargs)

In [ ]:
densfuncs = [pdens.triaxial_single_angle_zvecpa,
             pdens.triaxial_single_angle_zvecpa_plusexpdisk,
             pdens.triaxial_single_cutoff_zvecpa,
             pdens.triaxial_single_cutoff_zvecpa_plusexpdisk,
             pdens.triaxial_broken_angle_zvecpa,
             pdens.triaxial_broken_angle_zvecpa_plusexpdisk,
            #]
             pdens.triaxial_double_broken_angle_zvecpa,
             pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk]
versions = '100w_1e4n'
fit_type = 'gse'
feh_range_fit = copy.deepcopy(feh_range_gse)

# section = make_table_section(densfuncs,fit_type,version,selec,feh_range_gse,hf_kwargs)
table = make_gse_structural_table(densfuncs,versions,feh_range_fit,fit_type,
                                  hf_kwargs)
with open('tables/table_gse_structural.txt','w') as f:
    f.write(table)

In [ ]:
densfuncs = [pdens.triaxial_single_angle_zvecpa,
             pdens.triaxial_single_angle_zvecpa_plusexpdisk,
             pdens.triaxial_single_cutoff_zvecpa,
             pdens.triaxial_single_cutoff_zvecpa_plusexpdisk,
             pdens.triaxial_broken_angle_zvecpa,
             pdens.triaxial_broken_angle_zvecpa_plusexpdisk,
            #]
             pdens.triaxial_double_broken_angle_zvecpa,
             pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk]
versions = 'accmask_50w_5e3n'
fit_type = 'all'
feh_range_fit = copy.deepcopy(feh_range_all)


# section = make_table_section(densfuncs,fit_type,version,selec,feh_range_gse,hf_kwargs)
table = make_halo_structural_table(densfuncs,versions,feh_range_fit,fit_type,
                                   hf_kwargs)
with open('tables/table_halo_structural.txt','w') as f:
    f.write(table)

## Make the likelihood table

In [ ]:
densfuncs = [pdens.triaxial_single_angle_zvecpa,
             pdens.triaxial_single_angle_zvecpa_plusexpdisk,
             pdens.triaxial_single_cutoff_zvecpa,
             pdens.triaxial_single_cutoff_zvecpa_plusexpdisk,
             pdens.triaxial_broken_angle_zvecpa,
             pdens.triaxial_broken_angle_zvecpa_plusexpdisk,
            #]
             pdens.triaxial_double_broken_angle_zvecpa,
             pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk]
versions = '100w_1e4n'
fit_type = 'gse'
ml_type = 'post'
feh_range_fit = copy.deepcopy(feh_range_gse)


# section = make_table_section(densfuncs,fit_type,version,selec,feh_range_gse,hf_kwargs)
table = make_gse_likelihood_table(densfuncs,versions,feh_range_fit,fit_type,
                              ml_type,hf_kwargs)
with open('tables/table_gse_likelihood.txt','w') as f:
    f.write(table)

In [ ]:
densfuncs = [pdens.triaxial_single_angle_zvecpa,
             pdens.triaxial_single_angle_zvecpa_plusexpdisk,
             pdens.triaxial_single_cutoff_zvecpa,
             pdens.triaxial_single_cutoff_zvecpa_plusexpdisk,
             pdens.triaxial_broken_angle_zvecpa,
             pdens.triaxial_broken_angle_zvecpa_plusexpdisk,
            #]
             pdens.triaxial_double_broken_angle_zvecpa,
             pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk]
versions = 'accmask_50w_5e3n'
fit_type = 'all'
ml_type = 'post'
feh_range_fit = copy.deepcopy(feh_range_all)

# section = make_table_section(densfuncs,fit_type,version,selec,feh_range_gse,hf_kwargs)
table = make_halo_likelihood_table(densfuncs,versions,feh_range_fit,fit_type,
                              ml_type,hf_kwargs)
with open('tables/table_halo_likelihood.txt','w') as f:
    f.write(table)